In [1]:
import pandas as pd
from os import path

In [2]:
file_metadata = pd.read_csv('../data/ENA_PRJEB19856_metadata.tsv', sep='\t')
file_metadata.head()

,study_accession,sample_accession,experiment_accession,run_accession,tax_id,scientific_name,fastq_ftp,submitted_ftp,sra_ftp
0,PRJEB19856,SAMEA103892433,ERX1933710,ERR1873185,410658,soil metagenome,ftp.sra.ebi.ac.uk/vol1/fastq/ERR187/005/ERR187...,ftp.sra.ebi.ac.uk/vol1/run/ERR187/ERR1873185/A...,NaN
1,PRJEB19856,SAMEA103892434,ERX1933711,ERR1873186,410658,soil metagenome,ftp.sra.ebi.ac.uk/vol1/fastq/ERR187/006/ERR187...,ftp.sra.ebi.ac.uk/vol1/run/ERR187/ERR1873186/A...,NaN
2,PRJEB19856,SAMEA103892435,ERX1933712,ERR1873187,410658,soil metagenome,ftp.sra.ebi.ac.uk/vol1/fastq/ERR187/007/ERR187...,ftp.sra.ebi.ac.uk/vol1/run/ERR187/ERR1873187/A...,NaN
3,PRJEB19856,SAMEA103892436,ERX1933713,ERR1873188,410658,soil metagenome,ftp.sra.ebi.ac.uk/vol1/fastq/ERR187/008/ERR187...,ftp.sra.ebi.ac.uk/vol1/run/ERR187/ERR1873188/A...,NaN
4,PRJEB19856,SAMEA103892437,ERX1933714,ERR1873189,410658,soil metagenome,ftp.sra.ebi.ac.uk/vol1/fastq/ERR187/009/ERR187...,ftp.sra.ebi.ac.uk/vol1/run/ERR187/ERR1873189/A...,NaN


In [3]:
colnames = 'sample-id,forward-absolute-filepath,reverse-absolute-filepath'.split(',')
manifest_dict = {
    'sample-id': [],
    'forward-absolute-filepath': [],
    'reverse-absolute-filepath': [],
}

for idx, row_df in file_metadata.iterrows():
    ftp_paths = row_df.submitted_ftp.split(';')
    if len(ftp_paths) != 2:
        continue 
        
    fname_fwd = path.join('$PWD/data/', ftp_paths[0].split('/')[-1])
    fname_rev = path.join('$PWD/data/', ftp_paths[1].split('/')[-1])
    
    manifest_dict['sample-id'].append(row_df.run_accession)
    manifest_dict['forward-absolute-filepath'].append(fname_fwd)
    manifest_dict['reverse-absolute-filepath'].append(fname_rev)

In [4]:
manifest_df = pd.DataFrame(manifest_dict)

# Drop the last row since it appears to be a duplicate.
manifest_df.set_index('sample-id', inplace=True)
manifest_df.drop(axis=0, index='ERR2603003', inplace=True)
manifest_df.reset_index(inplace=True)

# Save file as csv for input to DADA2
manifest_df.to_csv('../data/manifest.tsv', sep='\t', index=False)

In [5]:
manifest_df.shape

(234, 3)